# PyTrees: Basic Example

This example is a Behavior Tree implementation of "Lab 0", replacing the finite state machine with a sequence of actions.

Read the documentation on https://py-trees.readthedocs.io/en/release-2.1.x/introduction.html

In [1]:
#!pip3 install py_trees

In [2]:
import py_trees
from controller import Robot
from controller import DistanceSensor, Motor, PositionSensor

In [3]:
class Waiting(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(Waiting, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [Waiting::setup()]" % self.name)
        distance_sensor.enable(timestep)

    def initialise(self):
        self.logger.debug("  %s [Waiting::initialise()]" % self.name)

    def update(self):
        self.logger.debug("  %s [Waiting::update()]" % self.name)
        if distance_sensor.getValue() < 500:
            self.feedback_message = "Can detected!"
            return py_trees.common.Status.SUCCESS # advance to Grasping
        else:
            return py_trees.common.Status.RUNNING
        
    def terminate(self, new_status):
        self.logger.debug("  %s [Waiting::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))

In [4]:
class Grasping(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(Grasping, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [Grasping::setup()]" % self.name)
        for motor in ur_motors:
            motor.setVelocity(speed)

    def initialise(self):
        self.counter=8
        for motor in hand_motors:
            motor.setPosition(0.85)
        self.logger.debug("  %s [Grasping::initialise()]" % self.name)

    def update(self):
        self.logger.debug("  %s [Grasping::update()]" % self.name)
        if(self.counter<=0):
            self.feedback_message = "Hand is closed."
            return py_trees.common.Status.SUCCESS # advance to Rotating
        else:
            self.counter=self.counter-1
            return py_trees.common.Status.RUNNING
        
    def terminate(self, new_status):
        self.logger.debug("  %s [Grasping::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))

In [5]:
class Rotating(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(Rotating, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [Rotating::setup()]" % self.name)
        position_sensor.enable(timestep)

    def initialise(self):
        for ind, motor in enumerate(ur_motors):
            motor.setPosition(target_positions[ind])
        self.logger.debug("  %s [Rotating::initialise()]" % self.name)

    def update(self):
        self.logger.debug("  %s [Rotating::update()]" % self.name)   
        if position_sensor.getValue() < -2.3:
            self.feedback_message = "Releasing can"
            for motor in hand_motors:
                motor.setPosition(motor.getMinPosition())
            return py_trees.common.Status.SUCCESS # advance to Releasing
        else:
            return py_trees.common.Status.RUNNING
                
    def terminate(self, new_status):
        self.logger.debug("  %s [Rotating::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))

In [6]:
class Releasing(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(Releasing, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [Releasing::setup()]" % self.name)

    def initialise(self):
        self.counter=8
        self.logger.debug("  %s [Releasing::initialise()]" % self.name)

    def update(self):
        self.logger.debug("  %s [Releasing::update()]" % self.name)   
        if self.counter<=0:
            self.feedback_message = "Hand is open."
            return py_trees.common.Status.SUCCESS # advance to rotate back    
        else:
            self.counter=self.counter-1
            return py_trees.common.Status.RUNNING
            
    def terminate(self, new_status):
        self.logger.debug("  %s [Releasing::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))

In [7]:
class RotatingBack(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(RotatingBack, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [RotatingBack::setup()]" % self.name)
        position_sensor.enable(timestep)

    def initialise(self):
        self.logger.debug("  %s [RotatingBack::initialise()]" % self.name)
        for motor in ur_motors:
            motor.setPosition(0.0)

    def update(self):
        self.logger.debug("  %s [RotatingBack::update()]" % self.name)
      
        if position_sensor.getValue() > -0.1:
            self.feedback_message = "Waiting can"
            return py_trees.common.Status.SUCCESS # advance to Waiting
        else:
            return py_trees.common.Status.RUNNING
                
    def terminate(self, new_status):
        self.logger.debug("  %s [RotatingBack::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))

In [ ]:
robot = Robot()
timestep = 32

target_positions = [-1.88, -2.14, -2.38, -1.51]
speed = 1.0

hand_motors = [robot.getDevice("finger_1_joint_1"),
               robot.getDevice("finger_2_joint_1"),
               robot.getDevice("finger_middle_joint_1")]
ur_motors = [robot.getDevice("shoulder_lift_joint"),
             robot.getDevice("elbow_joint"),
             robot.getDevice("wrist_1_joint"),
             robot.getDevice("wrist_2_joint")]
distance_sensor = robot.getDevice("distance sensor")
position_sensor = robot.getDevice("wrist_1_joint_sensor")


py_trees.logging.level = py_trees.logging.Level.DEBUG

We can now program the robot by composing a tree of appropriate behaviors. In this example, we create a "sequence" that consists of waiting, grasping, rotating, releasing, and rotating back states. 

In [ ]:
root = py_trees.composites.Sequence("Sequence")
root.add_child(Waiting(name="Waiting"))
root.add_child(Grasping(name="Grasping"))
root.add_child(Rotating(name="Rotating"))
root.add_child(Releasing(name="Releasing"))
root.add_child(RotatingBack(name="Rotating Back"))

#py_trees.display.render_dot_tree(root)
#import matplotlib.pyplot as plt
#import cv2
#%matplotlib inline
#img = cv2.imread('sequence.png')
#plt.figure(figsize = (20, 20))
#plt.imshow(img)

root.setup_with_descendants()

while robot.step(timestep) != -1:
    root.tick_once()
    

In [ ]:
root.setup_with_descendants()

In [ ]:
for i in range(10):
    root.tick_once()
    robot.step(timestep)